In [28]:

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [29]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [30]:
tf.__version__

'2.7.0'

In [31]:
x_train = train.drop('label', axis=1, inplace=False)
y_train = train['label']

In [32]:
x_train = x_train.values.reshape(-1, 28, 28, 1)
y_train = y_train.values.reshape(-1, 1)

In [33]:
'''scaler = MinMaxScaler()
scaler.fit(x_train)
x_data_norm = scaler.transform(x_train)
train_test_split(x_data_norm, y_train, test_size=0.3, random_state=0)'''

'scaler = MinMaxScaler()\nscaler.fit(x_train)\nx_data_norm = scaler.transform(x_train)\ntrain_test_split(x_data_norm, y_train, test_size=0.3, random_state=0)'

In [34]:
model = Sequential()
model.add(Conv2D())
model.add(Flatten(input_shape=(28, 28, 1)))
model.add(Dense(256, activation='relu'))
Dropout(0.2)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

In [35]:
'''model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(10, activation='softmax'))'''

"model = Sequential()\nmodel.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)))\nmodel.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)))\nmodel.add(MaxPool2D(pool_size=(2, 2), strides=2))\nmodel.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))\nmodel.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))\nmodel.add(MaxPool2D(pool_size=(2, 2), strides=2))\nmodel.add(Dropout(rate=0.25))\nmodel.add(Flatten())\nmodel.add(Dense(1024, activation='relu'))\nmodel.add(Dropout(rate=0.25))\nmodel.add(Dense(10, activation='softmax'))"

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               200960    
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dense_8 (Dense)             (None, 10)                1290      
                                                                 
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


In [37]:
batch_size = 128
epochs = 100

In [38]:
model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy']
)

In [39]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=10)]
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=callbacks 
                )
print(model.evaluate(x_train, y_train))


Epoch 1/100
263/263 [==============================] - 1s 3ms/step - loss: 3.8835 - accuracy: 0.8554 - val_loss: 1.1037 - val_accuracy: 0.9077
Epoch 2/100
263/263 [==============================] - 1s 2ms/step - loss: 0.6141 - accuracy: 0.9323 - val_loss: 0.6039 - val_accuracy: 0.9299
Epoch 3/100
263/263 [==============================] - 1s 2ms/step - loss: 0.3341 - accuracy: 0.9527 - val_loss: 0.5320 - val_accuracy: 0.9395
Epoch 4/100
263/263 [==============================] - 1s 2ms/step - loss: 0.2362 - accuracy: 0.9624 - val_loss: 0.5009 - val_accuracy: 0.9443
Epoch 5/100
263/263 [==============================] - 1s 2ms/step - loss: 0.1686 - accuracy: 0.9695 - val_loss: 0.4845 - val_accuracy: 0.9477
Epoch 6/100
263/263 [==============================] - 1s 2ms/step - loss: 0.1424 - accuracy: 0.9733 - val_loss: 0.4509 - val_accuracy: 0.9462
Epoch 7/100
263/263 [==============================] - 1s 2ms/step - loss: 0.1231 - accuracy: 0.9762 - val_loss: 0.3895 - val_accuracy: 0.9552